# Synthesize Responses from the Retrieved Contexts

## Step 0: Environment Setup

In [1]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index, get_entities,embed_entity, vector_search,enhanced_chunk_finder
import pandas as pd
from conn import connect2Googlesheet
from openai import OpenAI
from synthlibs import graph_retriever, parse_relationships, final_context_builder,synthesize_response, compute_accuracy_with_total_vocab, update_neutral_papers

# plot figures in high resolution
%config InlineBackend.figure_format = 'retina'
# Force reload of the .env file
try:
    load_dotenv()
    print("The .env file has been loaded")
except Exception as e:
    print("The .env file has not been loaded. Reported error: ", e)

The .env file has been loaded


In [2]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD"),
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")

# Check if the entities index exists
index_name = "entities"
query = "SHOW INDEXES YIELD name, type WHERE type = 'VECTOR' AND name = $index_name"

result = graph.query(query, params={"index_name": index_name})
if result:
    print("The 'entities' index already exists.")
else:
    create_vector_index(graph, "entities")

Connected to Neo4j database successfully.
The 'entities' index already exists.


## Step 1: Connect to Google Sheet

In [3]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
Q_worksheet = spreadsheet.get_worksheet(2)
# Select the worksheet: revised summary
Sum_worksheet = spreadsheet.get_worksheet(7)

# Get all records as a list of dictionaries
ques_data = Q_worksheet.get_all_records()
sum_data = Sum_worksheet.get_all_records()

# Convert to Pandas DataFrame
df_MedQ = pd.DataFrame(ques_data)
df_MedSum = pd.DataFrame(sum_data)

# Display the first few rows of both DataFrames
display(df_MedQ.head())
display(df_MedSum.head())

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blo...,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated wit...,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does...",Maybe this question: In patients with moderate...,fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have ...,Local question (not sure if this is the aim of...,fixed,Wrong concept since PEEP by itself is mandator...,Does the use of neuromuscular blockers in pati...
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservat...",Local question (not sure if this is the aim of...,fixed,Check if studies defined conservative by CVP <...,


,Type,Doc No,Topic,Name,Study Design and Methodology,Study Population,Interventions,Comparator,Outcomes,Strengths and Weaknesses,Key Findings and Conclusion
0,Ground Truth,24,ARDS,ENB,- Systematic review and meta-analysis\n- Searc...,- Patients with moderate to severe ARDS\n- Acr...,- 48-hour infusion of neuromuscular blocking a...,- Placebo or usual care\n- Control groups rece...,Primary Outcome:\n- ICU mortality\n\nSecondary...,Strengths:\n- Comprehensive literature search\...,Key Findings:\n- Reduced ICU mortality with NM...
1,Synthesis,,ARDS,ENB,- Meta-analysis of randomized controlled trials.,,,,,,The study associated the use of neuromuscular ...
2,Ground Truth,,ARDS,ACURASYS,"- Multicenter, randomized, double-blind, place...",The study enrolled 340 patients from 20 ICUs i...,- Cisatracurium Group:\n - 3-ml rapid IV infu...,- Placebo group receiving identical IV infusio...,Primary Outcome:\n- 90-day in-hospital mortali...,Strengths:\n- Multicenter design\n- Blinded ra...,Key Findings:\n- Adjusted 90-day survival impr...
3,Synthesis,,ARDS,ACURASYS,"- Multicenter, double-blind trial\n- Randomly ...",- 340 patients,- Cisatracurium besylate or placebo for 48 hours.,,- Reduced 90-day in-hospital mortality with ci...,,Key Findings:\nCisatracurium group had lower m...
4,Ground Truth,,ARDS,TOF-ARDS,"- Prospective, open-label study\n- Conducted i...",- 30 patients with moderate-to-severe ARDS\n- ...,- Continuous cisatracurium infusion\n- Initial...,- Theoretical dosage from ACURASYS study proto...,Primary Outcome:\n- Cisatracurium consumption\...,Strengths:\n- Novel approach to NMBA managemen...,Key Findings:\n- 60% reduction in overall cisa...


## Step 2: Build Context before Synthesis

In [4]:
test_question = df_MedQ["Question"][0]
context = graph_retriever(graph, test_question, max_depth=2)
formatted_relations = parse_relationships(context)
# print(formatted_relations)

In [5]:
final_context, filenames = final_context_builder(graph, test_question, method="hybrid")
# res = get_firstk_chunks(graph, filenames, firstk=3)
print(final_context)

Query: Does early administration of neuromuscular blocking agents increases the ventilator free days?
Found 19 unique relationships:
- Cochrane Library -> searched_for -> Neuromuscular blocking agents
- Embase -> searched_for -> Neuromuscular blocking agents
- Neuromuscular blocking agents -- contains -- Relevant articles
- Neuromuscular blocking agents -- searched_in -- literature search
- Neuromuscular blocking agents -- used_for -- literature search
- PubMed -> searched_for -> Neuromuscular blocking agents
- Neuromuscular blocking agents -- searched_in -- TTM2-trial
- Neuromuscular blocking agents -> includes -> Rocuronium
- Neuromuscular blocking agents -> includes -> Atracurium
- cisatracurium -> has_more -> ventilator-free days
- ventilator-free days -- used_agent -- ACURASYS trial
- ventilator-free days -- has_difference -- 28-day mortality
- ventilator-free days -- compared -- Acurasys
- ventilator-free days -- affects -- 90-day mortality
- ventilator-free days -- used_for -- A

## Step 3: Synthesize

### 3.1 Stance Synthesis

In [6]:
response_hy = synthesize_response(
    graph,
    test_question,
    synth_type="stance_synthesis",
    method="hybrid",
    model="gpt-4o-mini",
)
# response_vec = synthesize_response(
#     graph,
#     test_question,
#     synth_type="stance_synthesis",
#     method="vector",
#     model="gpt-3.5-turbo",
# )
# response_gr= synthesize_response(graph, test_question, method="graph", model="gpt-3.5-turbo")

In [7]:
print("Hybrid Response: ", response_hy.choices[0].message.content.strip())
# print("Vector Response: ", response_vec.choices[0].message.content.strip())
# print("Graph Response: ", response_gr.choices[0].message.content.strip())

Hybrid Response:  {
  "ACURASYS": "Supports",
  "FMWSCPARDS": "Supports",
  "ROSE": "Against",
  "ENB": "Neutral",
  "TOF-ARDS": "Supports",
  "ESCNBC": "Neutral",
  "DDS": "Neutral",
  "ETSDMV": "Supports",
  "RARDS": "Neutral",
  "HDDSICU": "Neutral"
}


#### Question 1 to 36

In [4]:
# Initialize an empty list to store the results
results_list = []

In [6]:
# Iterate over each question (0 to 35)
for idx in range(36):
    test_question = df_MedQ["Question"][idx]

    # Synthesize response
    response_hy = synthesize_response(
        graph,
        test_question,
        synth_type="stance_synthesis",
        method="hybrid",
        model="gpt-4o-mini",
    )

    # Extract and parse response
    stance_dict = response_hy.choices[0].message.content.strip()
    stance_dict = eval(stance_dict)  # Convert string dict to actual dict

    # Separate papers by stance
    support_papers = [
        paper for paper, stance in stance_dict.items() if stance == "Supports"
    ]
    against_papers = [
        paper for paper, stance in stance_dict.items() if stance == "Against"
    ]
    neutral_papers = [
        paper for paper, stance in stance_dict.items() if stance == "Neutral"
    ]

    # Append results
    results_list.append(
        {
            "Question Number": idx,
            "Question": test_question,
            "Support": ", ".join(support_papers),
            "Against": ", ".join(against_papers),
            "Neutral": ", ".join(neutral_papers),
        }
    )

[#C8A6]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.69.128.95', 7687)) (ResolvedIPv4Address(('34.69.128.95', 7687))): TimeoutError('The read operation timed out')


In [13]:
# Create DataFrame
df_results = pd.DataFrame(results_list)
df_results["Question Number"] = df_results["Question Number"] + 1

In [14]:
df_results

,Question Number,Question,Support,Against,Neutral
0,1,Does early administration of neuromuscular blo...,"ACURASYS, FMWSCPARDS, ETSDMV, TOF-ARDS",ROSE,"ESCNBC, RARDS, DDS, HDDSICU"
1,2,Do patients with severe ARDS being treated wit...,,"ROSE, NBSARDS, ACURASYS, LSPA","BMIMSARDS, EDPMARDSLPMV, PROSEVA, TOF-ARDS, CE..."
2,3,"In patients with moderate to severe ARDS, does...","NBSARDS, ACURASYS, LSPA","ROSE, ESCNBC, ESCPARDS","BMIMSARDS, PROSEVA, CCCARDS, CEIIUPPSARDS"
3,4,Do patients with moderate-to-severe ARDS have ...,"ACURASYS, TOF-ARDS",ROSE,"ENB, NBSARDS, BMIMSARDS, EDPMARDSLPMV, CEIIUPP..."
4,5,"Among patients with ALI/ARDS, does a conservat...","FMWSCPARDS, PAC-in-FACTT",,"NBSARDS, RARDS, DTNP, VMOCA"
5,6,Does a conservative fluid management strategy ...,PAC-in-FACTT,,FACTT
6,7,Does mechanical ventilation with a lower tidal...,"ARDSNet, ENB, NBSARDS, EDPMARDSLPMV, CEIIUPPSA...",,"RARDS, HARDST, VMOCA, ETSDMV"
7,8,Does mechanical ventilation with a lower tidal...,"NBSARDS, RARDS, FMWSCPARDS, EDPMARDSLPMV, HARD...",,"BMIMSARDS, ETSDMV, VMOCA"
8,9,Prone positioning sessions in patients with se...,"ACURASYS, PROSEVA, CEIIUPPSARDS",ROSE,"NBSARDS, BMIMSARDS, CCCARDS, EDPMARDSLPMV, LSP..."
9,10,Does early application of high frequency oscil...,,"ARDSNet, OSCILLATE","BMIMSARDS, CEIIUPPSARDS, NBSARDS, HARDST, EDPM..."


#### Save the stance model result data once

In [15]:
# Save to CSV
df_results.to_csv("./outputs/stance_synthesis_results.csv", index=False)

print("✅ Saved results to stance_synthesis_results.csv")

✅ Saved results to stance_synthesis_results.csv


#### Load the stance model result data

In [18]:
# Load the synthesis results
df_results = pd.read_csv("./outputs/stance_synthesis_results.csv")

#### Add All Neutral Papers

In [26]:
# Example: Retrieve total_vocab_set from a Google Spreadsheet
total_vocab = spreadsheet.get_worksheet(1).col_values(3)[1:]  # Skip the header row
total_vocab_set = set(
    [s.strip() for s in total_vocab if s.strip() != ""]
)  # Remove empty and whitespace-only entries

In [27]:
# Apply the update function to your DataFrame
df_updated = df_results.apply(
    lambda row: update_neutral_papers(row, total_vocab_set), axis=1
)

### 3.1.1 Synthesis Evaluation

In [5]:
# Load the stance spreadsheet
stance_worksheet = spreadsheet.get_worksheet(4)

# Get all records as a list of dictionaries
stance_data = stance_worksheet.get_all_records()

# Convert to Pandas DataFrame
df_Stance = pd.DataFrame(stance_data)

# Display the first few rows of DataFrame
display(df_Stance.head())

,Done by,Document,StanceQ1,StanceQ2,StanceQ3,StanceQ4,StanceQ5,StanceQ6,StanceQ7,StanceQ8,...,StanceQ27,StanceQ28,StanceQ29,StanceQ30,StanceQ31,StanceQ32,StanceQ33,StanceQ34,StanceQ35,StanceQ36
0,,ACURASYS,1,-1,,,,,,,...,,,,,,,,,,
1,,ROSE,,1,-1,-1,,,,,...,,,,,,,,,,
2,,FACTT,,,,,1,-1,,,...,,,,,,,,,,
3,,ARDSNet,,,,,,,1,1,...,,,,,,,,,,
4,,PROSEVA,,,,,,,,,...,,,,,,,,,,


#### Save the stance annotation file once

In [32]:
# Store the stance spreadsheet
df_Stance.to_csv("./outputs/stance_annotations.csv", index=False)

#### Load the stored stance annotation data

In [6]:
# Load the stance annotation data
df_Stance = pd.read_csv("./outputs/stance_annotations.csv")

In [7]:
# Create a list to store the results for each question
results = []

# Loop through each question column from StanceQ1 to StanceQ36
for i in range(1, 37):
    col = f"StanceQ{i}"
    # Support: rows where the value equals 1
    support = df_Stance.loc[df_Stance[col] == 1, "Document"].dropna().tolist()
    # Against: rows where the value equals -1
    against = df_Stance.loc[df_Stance[col] == -1, "Document"].dropna().tolist()
    # Neutral: rows where the value is 0 or is missing (NaN)
    neutral = (
        df_Stance.loc[(df_Stance[col].isna()) | (df_Stance[col] == 0), "Document"]
        .dropna()
        .tolist()
    )

    results.append(
        {
            "Question Number": i,
            "Support": support,
            "Against": against,
            "Neutral": neutral,
        }
    )

In [8]:
# Create a DataFrame from the results list
result_stance = pd.DataFrame(results)

# Convert list columns to comma-separated strings without brackets.
for col in ["Support", "Against", "Neutral"]:
    result_stance[col] = result_stance[col].apply(lambda x: ", ".join(x) if x else "")

# Print the resulting DataFrame
result_stance

,Question Number,Support,Against,Neutral
0,1,"ACURASYS, NBSARDS, TOF-ARDS","LSPA, ENB, ESCNBC","ROSE, FACTT, ARDSNet, PROSEVA, OSCILLATE, ADRE..."
1,2,ROSE,"ACURASYS, LSPA, TOF-ARDS","FACTT, ARDSNet, PROSEVA, OSCILLATE, ADRENAL, A..."
2,3,ENB,"ROSE, ESCNBC","ACURASYS, FACTT, ARDSNet, PROSEVA, OSCILLATE, ..."
3,4,,"ROSE, ESCNBC","ACURASYS, FACTT, ARDSNet, PROSEVA, OSCILLATE, ..."
4,5,"FACTT, ARDSSRDRFMS, FMWSCPARDS",PAC-in-FACTT,"ACURASYS, ROSE, ARDSNet, PROSEVA, OSCILLATE, A..."
5,6,ARDSSRDRFMS,"FACTT, PAC-in-FACTT","ACURASYS, ROSE, ARDSNet, PROSEVA, OSCILLATE, A..."
6,7,"ARDSNet, DIHRFARDSNET",DLASPEEPvsARDSNET,"ACURASYS, ROSE, FACTT, PROSEVA, OSCILLATE, ADR..."
7,8,ARDSNet,DLASPEEPvsARDSNET,"ACURASYS, ROSE, FACTT, PROSEVA, OSCILLATE, ADR..."
8,9,PROSEVA,PPATDSHMA,"ACURASYS, ROSE, FACTT, ARDSNet, OSCILLATE, ADR..."
9,10,,OSCILLATE,"ACURASYS, ROSE, FACTT, ARDSNet, PROSEVA, ADREN..."


#### Save the merged data once

In [13]:
merged_stance = pd.merge(df_results, result_stance, on="Question Number")

merged_stance.to_csv("./outputs/merged_stance_final.csv", index=False)

#### Load the merged data

In [9]:
df_merged = pd.read_csv("./outputs/merged_stance.csv")

#### 3.1.1 Accuracy Evaluation

In [21]:
df_updated

,Question Number,Question,Support,Against,Neutral
0,1,Does early administration of neuromuscular blo...,"ACURASYS, ETSDMV, FMWSCPARDS, TOF-ARDS",ROSE,"ADRENAL, AID-ICU, ALBIOS, AMTD, ANNANE, APROCC..."
1,2,Do patients with severe ARDS being treated wit...,,"ACURASYS, LSPA, NBSARDS, ROSE","ADRENAL, AID-ICU, ALBIOS, AMTD, ANNANE, APROCC..."
2,3,"In patients with moderate to severe ARDS, does...","ACURASYS, LSPA, NBSARDS","ESCNBC, ESCPARDS, ROSE","ADRENAL, AID-ICU, ALBIOS, AMTD, ANNANE, APROCC..."
3,4,Do patients with moderate-to-severe ARDS have ...,"ACURASYS, TOF-ARDS",ROSE,"ADRENAL, AID-ICU, ALBIOS, AMTD, ANNANE, APROCC..."
4,5,"Among patients with ALI/ARDS, does a conservat...","FMWSCPARDS, PAC-in-FACTT",,"ACURASYS, ADRENAL, AID-ICU, ALBIOS, AMTD, ANNA..."
5,6,Does a conservative fluid management strategy ...,PAC-in-FACTT,,"ACURASYS, ADRENAL, AID-ICU, ALBIOS, AMTD, ANNA..."
6,7,Does mechanical ventilation with a lower tidal...,"ARDSNet, CEIIUPPSARDS, EDPMARDSLPMV, ENB, LSPA...",,"ACURASYS, ADRENAL, AID-ICU, ALBIOS, AMTD, ANNA..."
7,8,Does mechanical ventilation with a lower tidal...,"CEIIUPPSARDS, EDPMARDSLPMV, FMWSCPARDS, HARDST...",,"ACURASYS, ADRENAL, AID-ICU, ALBIOS, AMTD, ANNA..."
8,9,Prone positioning sessions in patients with se...,"ACURASYS, CEIIUPPSARDS, PROSEVA",ROSE,"ADRENAL, AID-ICU, ALBIOS, AMTD, ANNANE, APROCC..."
9,10,Does early application of high frequency oscil...,,"ARDSNet, OSCILLATE","ACURASYS, ADRENAL, AID-ICU, ALBIOS, AMTD, ANNA..."


In [22]:
# Merge the updated DataFrame with the stance results
merged_stance_final = pd.merge(df_updated, result_stance, on="Question Number")

In [23]:
merged_stance_final

,Question Number,Question,Support_x,Against_x,Neutral_x,Support_y,Against_y,Neutral_y
0,1,Does early administration of neuromuscular blo...,"ACURASYS, ETSDMV, FMWSCPARDS, TOF-ARDS",ROSE,"ADRENAL, AID-ICU, ALBIOS, AMTD, ANNANE, APROCC...","ACURASYS, NBSARDS, TOF-ARDS","LSPA, ENB, ESCNBC","ROSE, FACTT, ARDSNet, PROSEVA, OSCILLATE, ADRE..."
1,2,Do patients with severe ARDS being treated wit...,,"ACURASYS, LSPA, NBSARDS, ROSE","ADRENAL, AID-ICU, ALBIOS, AMTD, ANNANE, APROCC...",ROSE,"ACURASYS, LSPA, TOF-ARDS","FACTT, ARDSNet, PROSEVA, OSCILLATE, ADRENAL, A..."
2,3,"In patients with moderate to severe ARDS, does...","ACURASYS, LSPA, NBSARDS","ESCNBC, ESCPARDS, ROSE","ADRENAL, AID-ICU, ALBIOS, AMTD, ANNANE, APROCC...",ENB,"ROSE, ESCNBC","ACURASYS, FACTT, ARDSNet, PROSEVA, OSCILLATE, ..."
3,4,Do patients with moderate-to-severe ARDS have ...,"ACURASYS, TOF-ARDS",ROSE,"ADRENAL, AID-ICU, ALBIOS, AMTD, ANNANE, APROCC...",,"ROSE, ESCNBC","ACURASYS, FACTT, ARDSNet, PROSEVA, OSCILLATE, ..."
4,5,"Among patients with ALI/ARDS, does a conservat...","FMWSCPARDS, PAC-in-FACTT",,"ACURASYS, ADRENAL, AID-ICU, ALBIOS, AMTD, ANNA...","FACTT, ARDSSRDRFMS, FMWSCPARDS",PAC-in-FACTT,"ACURASYS, ROSE, ARDSNet, PROSEVA, OSCILLATE, A..."
5,6,Does a conservative fluid management strategy ...,PAC-in-FACTT,,"ACURASYS, ADRENAL, AID-ICU, ALBIOS, AMTD, ANNA...",ARDSSRDRFMS,"FACTT, PAC-in-FACTT","ACURASYS, ROSE, ARDSNet, PROSEVA, OSCILLATE, A..."
6,7,Does mechanical ventilation with a lower tidal...,"ARDSNet, CEIIUPPSARDS, EDPMARDSLPMV, ENB, LSPA...",,"ACURASYS, ADRENAL, AID-ICU, ALBIOS, AMTD, ANNA...","ARDSNet, DIHRFARDSNET",DLASPEEPvsARDSNET,"ACURASYS, ROSE, FACTT, PROSEVA, OSCILLATE, ADR..."
7,8,Does mechanical ventilation with a lower tidal...,"CEIIUPPSARDS, EDPMARDSLPMV, FMWSCPARDS, HARDST...",,"ACURASYS, ADRENAL, AID-ICU, ALBIOS, AMTD, ANNA...",ARDSNet,DLASPEEPvsARDSNET,"ACURASYS, ROSE, FACTT, PROSEVA, OSCILLATE, ADR..."
8,9,Prone positioning sessions in patients with se...,"ACURASYS, CEIIUPPSARDS, PROSEVA",ROSE,"ADRENAL, AID-ICU, ALBIOS, AMTD, ANNANE, APROCC...",PROSEVA,PPATDSHMA,"ACURASYS, ROSE, FACTT, ARDSNet, OSCILLATE, ADR..."
9,10,Does early application of high frequency oscil...,,"ARDSNet, OSCILLATE","ACURASYS, ADRENAL, AID-ICU, ALBIOS, AMTD, ANNA...",,OSCILLATE,"ACURASYS, ROSE, FACTT, ARDSNet, PROSEVA, ADREN..."


In [24]:
# Apply the accuracy computation function to each row
df_accuracy_ = merged_stance_final.apply(
    lambda row: compute_accuracy_with_total_vocab(row, total_vocab_set), axis=1
)

In [25]:
# Check the result for 36 questions
df_accuracy

,Correct Predictions,Total Papers,Accuracy
0,94.0,112.0,0.839286
1,98.0,112.0,0.875000
2,96.0,112.0,0.857143
3,98.0,112.0,0.875000
4,98.0,112.0,0.875000
5,98.0,112.0,0.875000
6,94.0,112.0,0.839286
7,92.0,112.0,0.821429
8,97.0,112.0,0.866071
9,101.0,112.0,0.901786


In [15]:
df_final.to_csv("./outputs/accuracy_results.csv", index=False)

### 3.2 PICO Synthesis

In [ ]:
response_hy = synthesize_response(
    graph,
    test_question,
    synth_type="PICO_synthesis",
    method="hybrid",
    model="gpt-3.5-turbo",
)

"""
response_vec = synthesize_response(
    graph,
    test_question,
    synth_type="PICO_synthesis",
    method="vector",
    model="gpt-3.5-turbo",
)
"""

'\nresponse_vec = synthesize_response(\n    graph,\n    test_question,\n    synth_type="PICO_synthesis",\n    method="vector",\n    model="gpt-3.5-turbo",\n)\n'

In [ ]:
print("Hybrid Response: ", response_hy.choices[0].message.content.strip())
# print("Vector Response: ", response_vec.choices[0].message.content.strip())

Hybrid Response:  Based on the articles provided, the studies can be grouped as follows:

### Supporting the Claim:
1. **Study 1: PLMALDHSS**
   - **Authors:** Romain Pirracchio et al.
   - **Year:** 2023
   - **Title:** Patient-Level Meta-Analysis of Low-Dose Hydrocortisone in Adults with Septic Shock
   - **Journal:** NEJM Evidence
   - **Research Question:** To evaluate the effect of hydrocortisone infusion on 90-day mortality in patients with septic shock.
   - **Study Design:** Meta-analysis of patient-level data from septic shock trials.
   - **Population Size:** 528 participants across 17 trials.
   - **Intervention:** Hydrocortisone infusion.
   - **Primary Outcome:** 90-day mortality was not significantly reduced by hydrocortisone.
   - **Key Findings:** Hydrocortisone did not result in lower 90-day mortality compared to placebo.
   - **Conclusion:** The study findings do not support the use of hydrocortisone to reduce 90-day mortality in septic shock patients.

### Refuting t

In [ ]:
response_hy = synthesize_response(
    graph,
    test_question,
    synth_type="PICO_synthesis",
    method="hybrid",
    model="gpt-3.5-turbo",
)

In [ ]:
print("Hybrid Response: ", response_hy.choices[0].message.content.strip())

Hybrid Response:  Based on the provided articles, the summaries are as follows:

### Supporting the Claim:
1. **Study 1**
   - **Authors:** Pirracchio et al.
   - **Year:** Not specified
   - **Title:** "Patient-Level Meta-Analysis of Low-Dose Hydrocortisone in Adults with Septic Shock"
   - **Journal:** Not specified
   - **Research Question:** Investigated the effect of hydrocortisone on 90-day all-cause mortality in patients with septic shock.
   - **Type of Study:** Meta-analysis
   - **Methodology:**
     - Pooled individual patient data from septic shock trials using hydrocortisone.
     - Primary outcome: 90-day all-cause mortality.
   - **Key Findings:**
     - Hydrocortisone vs. placebo RR for 90-day mortality was 0.93.
     - Hydrocortisone with fludrocortisone showed a RR of 0.86.
     - Hydrocortisone associated with more vasopressor-free days.
   - **Conclusion:** Hydrocortisone showed no significant reduction in 90-day mortality but improved vasopressor-free days.

### Re

In [ ]:
response_hy = synthesize_response(
    graph,
    test_question,
    synth_type="PICO_synthesis",
    method="hybrid",
    model="gpt-4o-mini",
)

In [ ]:
print("Hybrid Response: ", response_hy.choices[0].message.content.strip())

Hybrid Response:  Based on the provided articles, I have categorized the studies according to their findings regarding the claim that early administration of neuromuscular blocking agents (NMBAs) increases ventilator-free days (VFDs):

### Supporting the Claim
1. **Authors**: Lyu et al.
   - **Year**: 2021
   - **Title**: The effect of neuromuscular blocking agents uses in acute respiratory distress syndrome: a systematic review and meta-analysis of randomized controlled trials.
   - **Journal**: Minerva Anestesiologica
   - **Research Question**: Does the early administration of NMBAs improve ventilator-free days in patients with moderate to severe ARDS?
   - **Type of Study**: Meta-analysis of randomized controlled trials.
   - **Methodology**: Reviewed multiple RCTs comparing early NMBA use versus control in ARDS patients, focusing on VFDs.
   - **Population Size**: Data from numerous trials with a total of 1,953 patients.
   - **Key Findings**: Early NMBA infusion was associated wi